In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as datetime_price
import pytz
from dateutil.relativedelta import relativedelta
import time

# Insert the List of Users

In [2]:
user_df = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\Input\Top 20 Cryptocurrencies and Related TW Accounts.csv", encoding= 'unicode_escape')
user_df

,Coin,ID,Owner/Influence,Name,Screen_name,Status,Why did I choose this person?
0,Ethereum,ethereum,Vitalik Buterin,vitalik.eth,VitalikButerin,co-founder,he is also known in crypto world
1,BNB,binancecoin,Changpeng Zhao,CZ ??Binance,cz_binance,CEO,NaN
2,XRP,ripple,Brad Garlinghouse,Brad Garlinghouse,bgarlinghouse,CEO,NaN
3,ADA(Cardano),cardano,Charles Hoskinson,Charles Hoskinson,IOHK_Charles,CEO/co-founder,NaN
4,DOGE,dogecoin,Elon Musk_doge,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."
5,MATIC(Polygon),matic-network,Sandeep Nailwal,Sandeep | Polygon ????3??,sandeepnailwal,co-founder,he seems more active than other co-founders
6,DOT(Polkadot),polkadot,WEB3 Foundation,Web3 Foundation,Web3foundation,foundation,there is no account of CEO or sth
7,HEX,hex,Richard Heart,Richard Heart,RichardHeartWin,CEO,NaN
8,SHIB,shiba-inu,Elon Musk_shiba,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."
9,SOL,solana,Solana,Solana,solana,company,there is no account of CEO or sth


# Coin Data

# Getting Historical Price Data

In [3]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [4]:
"""
coingecko api will give price data like:
Get historical market data include price, market cap, and 24h volume within a range of timestamp (granularity auto) 
<b>Minutely data will be used for duration within 1 day, Hourly data will be used for duration between 1 day and 90 days,
Daily data will be used for duration above 90 days.</b>
"""

#datetime format -> (year, month, day, hour, minute, second)
endDate_price = datetime.datetime(2022, 12, 31, 23, 59, 59)
startDate_price = endDate_price - relativedelta(days = 401)
#endDate_price = datetime.datetime(2023, 1, 1, 23, 59, 59)
#startDate_price = datetime.datetime(2022, 12, 31, 0, 0, 0)

#endDate_price = datetime.datetime.now()
#startDate_price = endDate_price- datetime.timedelta(minutes = 10)

#startDate_price = endDate_price - relativedelta(days = 1) #days, months, years can be used to calculate


ts_endDate_price = datetime_price.timestamp(endDate_price)
ts_startDate_price = datetime_price.timestamp(startDate_price)

In [5]:
def get_price(coin_ids):
    
    result = pd.DataFrame(columns = ["ID", "timestamps", "date_time", "price", "market_caps", "volumes"])
    for x in coin_ids:
        
        current_coin_data = cg.get_coin_market_chart_range_by_id(id = x, vs_currency = 'usd', from_timestamp = ts_startDate_price, to_timestamp = ts_endDate_price)
        
        dummy_df = pd.DataFrame(current_coin_data["prices"], columns = ["timestamps", "price" ])
        dummy_marketcaps = pd.DataFrame(current_coin_data["market_caps"], columns = ["timestamps", "market_caps" ])
        dummy_volumes = pd.DataFrame(current_coin_data["total_volumes"], columns = ["timestamps", "volumes" ])
        
        dummy_df = dummy_df.join(dummy_marketcaps["market_caps"])
        dummy_df = dummy_df.join(dummy_volumes["volumes"])
        
        dummy_df["ID"] = x
        dummy_df["date_time"] = pd.to_datetime(dummy_df['timestamps']/1000, unit='s')
        
        dummy_df = dummy_df[["ID", "timestamps", "date_time", "price", "market_caps", "volumes"]]
        
        result = result.append(dummy_df)
        
    return result

In [6]:
price_data = get_price(user_df["ID"])

In [7]:
price_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6416 entries, 0 to 400
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           6416 non-null   object        
 1   timestamps   6416 non-null   object        
 2   date_time    6416 non-null   datetime64[ns]
 3   price        6416 non-null   float64       
 4   market_caps  6416 non-null   float64       
 5   volumes      6416 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 350.9+ KB


In [8]:
price_data.shape

(6416, 6)

In [9]:
price_data.isna().any().any()

False

In [10]:
price_data[price_data.duplicated(keep=False)].shape[0]

0

In [11]:
price_data.head()

,ID,timestamps,date_time,price,market_caps,volumes
0,ethereum,1637884800000,2021-11-26,4515.843300,5.350824e+11,2.024153e+10
1,ethereum,1637971200000,2021-11-27,4048.312844,4.794143e+11,2.927673e+10
2,ethereum,1638057600000,2021-11-28,4084.088486,4.839973e+11,1.650051e+10
3,ethereum,1638144000000,2021-11-29,4290.091863,5.083852e+11,1.740861e+10
4,ethereum,1638230400000,2021-11-30,4444.528280,5.273311e+11,2.015092e+10


In [12]:
price_data.to_excel(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\price_data.xlsx")
price_data.to_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\Input Data\price_data.csv")